In [1]:
import plotly.graph_objects as go
import numpy as np

In [14]:
def creer_maillage_3D(x_min : float, x_max : float, y_min : float, y_max : float, z_min : float, z_max : float, pas_du_maillage : float) -> list:
    """
    Entrées:

        - x_min : la valeur minimale en x du maillage (Exemple : x_min = -2.5). Le signe est quelconque et la valeur doit être différente de x_max.
        - y_min : la valeur minimale en y du maillage (Exemple : y_min = -2.5). Le signe est quelconque et la valeur doit être différente de y_max.
        - z_min : la valeur minimale en z du maillage (Exemple : z_min = -2.5). Le signe est quelconque et la valeur doit être différente de z_max.

        - x_max : la valeur maximale en x du maillage (Exemple : x_max = 2.5). Le signe est quelconque et la valeur doit être différente de x_min.
        - y_max : la valeur maximale en y du maillage (Exemple : y_max = 2.5). Le signe est quelconque et la valeur doit être différente de y_min.
        - z_max : la valeur maximale en z du maillage (Exemple : z_max = 2.5). Le signe est quelconque et la valeur doit être différente de z_min.

        - pas_du_maillage : distance séparant 2 points (non diagonnaux) du maillage. Il est de préférence un diviseur 
            de (x_max - x_min) et de (y_max - y_min) et (z_max - z_min). Éviter un pas égal à 0.3 ou quelconque multiple 
            de 0.3 ainsi que les nombres premiers par exemple, les coordonnées des points doivent être arrondies ce qui 
            peut poser problème pour les trouver par la suite avec la fonction find_valeur_point(). Sa valeur ne doit 
            pas être plus élevée que la moitié de la plus petite distance coordonnée_max - coordonnée_min des 3 axes.
        
    Sorties:
    
        - [X,Y,Z] : C'est une liste qui contient 3 listes de NP.ARRAYS 2D. 
            
            * len(X) = len(Y) = len(Z) = nb_points_entre_y_min_y_max.
            
            * X[0] ou Y[0] ou Z[0] représente le plan (xOz) pour y = y_min, X[1] ou Y[1] ou Z[1] pour y = y_min + pas_du_maillage et ainsi 
                de suite jusqu'à y = y_max. 
            
            * De plus, pour chaque NP.ARRAY de X, Y ou Z, X[y_fixé][:,0], Y[y_fixé][:,0], Z[y_fixé][:,0] représente les droites parallèles 
                à l'axe (Ox) pour y = y_fixé et z = z_min. De même, X[y_fixé][:,1], Y[y_fixé][:,1], Z[y_fixé][:,1] représente les droites 
                parallèles à l'axe (Ox) pour y = y_fixé et z = z_min + pas_du_maillage. 
            
            * On a également X[y_fixé][0,:], Y[y_fixé][0,:], Z[y_fixé][0,:] représente les droites parallèles à l'axe (Oz) pour y = y_fixé et 
                x = x_min. De même, X[y_fixé][1,:], Y[y_fixé][1,:], Z[y_fixé][1,:] représente les droites parallèles à l'axe (Oz) pour y = y_fixé 
                et x = x_min + pas_du_maillage.
            
    Explication:
    
        La fonction creer_maillage_3D() crée une maillage 3D de l'espace pour des limites et un pas donnés.
    """
    
    #Vérification de la cohérence des données.
    for min, max in zip([x_min,y_min,z_min],[x_max,y_max,z_max]):
        if min == max: #Sinon le maillage est soit: un plan, une droite ou un point ce qui ne nous intéresse pas ici.
            raise ValueError(f"Le maillage n'est pas 3D: {min} = {max}.")
        if min > max:
            min, max = max, min #Nécessaire pour avoir des np.linspace(min, max, nb_points) cohérents.
        if pas_du_maillage > float(abs(max-min)/2):
            raise ValueError("pas_du_maillage est trop grand.") #Le cas écheant, le maillage ne serait pas bien défini.

    #Création du maillage.
    longueur_maillage_x = abs(x_max - x_min)
    longueur_maillage_y = abs(y_max - y_min)
    longueur_maillage_z = abs(z_max - z_min)
    
    nb_points_entre_x_min_x_max = int((longueur_maillage_x) / (pas_du_maillage)) + 1
    nb_points_entre_y_min_y_max = int((longueur_maillage_y) / (pas_du_maillage)) + 1
    nb_points_entre_z_min_z_max = int((longueur_maillage_z) / (pas_du_maillage)) + 1

    axe_x = np.linspace(x_min, x_max, nb_points_entre_x_min_x_max)
    axe_y = np.linspace(y_min, y_max, nb_points_entre_y_min_y_max)
    axe_z = np.linspace(z_min, z_max, nb_points_entre_z_min_z_max)
    
    X,Y,Z = np.meshgrid(axe_x,axe_y,axe_z)
    
    #Permet de gérer les cas où pas_du_maillage engendre des points dont les coordonnées ont des décimales infinies comme 0.3.
    X = np.round(X, 1) 
    Y = np.round(Y, 1)
    Z = np.round(Z, 1)

    return [X,Y,Z]

cote = 10 #N'est pas une variable globale.
pas_du_maillage = 0.5 #N'est pas une variable globale.
X,Y,Z = creer_maillage_3D(-cote,cote,-cote,cote,-cote,cote,pas_du_maillage)
print(f"Le maillage est composé de {len(X)*X[0].shape[0]*X[0].shape[1]:,} points.")

Le maillage est composé de 68,921 points.


In [3]:
def trouver_signe_afficher_maillage_3D(valeur : float) -> str:
    """
    Cette fonction n'est utilisée qu'uniquement par la fonction afficher_maillage_3D(). Elle
    permet de renvoyer une string du signe de "valeur".
    """
    if valeur >= 0:
        return '+'
    return '-'

def calculer_opacite_afficher_maillage_3D(valeur : float, valeur_ref : float) -> float:
    """
    Cette fonction n'est utilisée qu'uniquement par la fonction afficher_maillage_3D(). Elle
    permet de renvoyer une valeur d'opacité en fonction de la position du slider.
    """
    if abs(valeur - valeur_ref) < 1:
        return 1
    if abs(valeur - valeur_ref) < 1.5:
        return 0.3
    if abs(valeur - valeur_ref) < 2.5:
        return 0.2
    else:
        return 0.1
    
def calcul_liste_arg_slider_afficher_maillage_3D(valeur_ref : int, precision_slider : int) -> list:
    """
    Cette fonction n'est utilisée qu'uniquement par la fonction afficher_maillage_3D(). Elle permet 
    de renvoyer une liste contenant les valeurs d'opacité en fonction de la position du slider.
    """
    liste = [0,0]
    for valeur in range(precision_slider):
        liste.append(calculer_opacite_afficher_maillage_3D(float(valeur) + 0.1, valeur_ref))
    return liste 

In [4]:
def afficher_maillage_3D(X : np.array, Y : np.array, Z : np.array, titre : str, grandeur = None, nom_grandeur  = None, unite = None, afficher_array = None) -> None:
    """
    Entrées:
    
        - X: C'est une liste de NP.ARRAYS 2D. Usuellement celle renvoyée par la fonction creer_maillage_3D().
        - Y: C'est une liste de NP.ARRAYS 2D. Usuellement celle renvoyée par la fonction creer_maillage_3D().
        - Z: C'est une liste de NP.ARRAYS 2D. Usuellement celle renvoyée par la fonction creer_maillage_3D().
        
        - afficher_array: C'est un booléen. Mettre en argument "True" si l'on souhaite avoir un aperçu des liste X, Y, Z. 
            "False" autrement.
        
        - titre: C'est une string. Correspond au titre affiché sur le plot lorsque "afficher_plot == True".
        
        - grandeur: C'est une liste de NP.ARRAYS 2D. Ses dimensions doivent être égales à celles du maillage.
            On peut par exemple y mettre la liste renvoyée par la fonction calcul_charge_tout_point_maillage().
        - nom_grandeur: C'est une string. Elle correspond à ce qui est écrit dans les boutons ou au titre 
            de la colorbar lorsque "afficher_plot == True". 
        - unite: C'est une string. Elle correspond à l'unité associée à "grandeur" lorsque "grandeur != None"
        
    Sortie:
    
        - None: C'est une méthode. La fonction ne renvoie rien.
        
    Explication:
    
        La fonction permet d'avoir un idée de ce à quoi ressemble notre maillage lorsqu'une grandeur est associée
        à chaque point (Le potentiel par exemple ou encore la charge). Elle permet aussi de visualiser les listes
        du maillage.
    """
    
    precision_slider = 50 #Plus le nombre est grand et plus la tolérance sur les valeurs diminue. C'est un int.
    if type(precision_slider) != int:
        raise ValueError("La précision du slider doit être un int.")
    
    couleur = "temps" #Choix de la couleur: 'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid','turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr', 'ylorrd', 'inferno'.
    
    methode_affichage = 'browser' #Doit être une string : "browser" ou "vscode". Affiche soit dans le navigateur, soit sur vscode.
     
    #Gère le cas d'une valeur affectée aux points.
    if grandeur is not None:
        fig = go.Figure()
        mask_charge_positive = grandeur.flatten() > 0 #Filtre les valeurs positives.
        mask_charge_negative = grandeur.flatten() <= 0 #Filtre les valeurs négatives ou nulles.
        
        #Création d'une trace affichant seulement les grandeurs positives. C'est la trace(1)
        fig.add_trace(go.Scatter3d(
            x=X.flatten()[mask_charge_positive], y=Y.flatten()[mask_charge_positive], z=Z.flatten()[mask_charge_positive],
            mode="markers",
            marker=dict(size=2, #Taille des points.
                        color=grandeur.flatten()[mask_charge_positive],
                        colorscale=couleur,
                        cmin = grandeur.min(),
                        cmax = grandeur.max(),
                        opacity=1, 
                        colorbar=dict(
                        title=f'{nom_grandeur} ({unite})', #Titre de la colorbar 
                        ticks="outside",  
                        tickvals=[grandeur.min(), grandeur.max()], #Plage des valeurs de la colorbar
                        ticktext=[f"{grandeur.min():{trouver_signe_afficher_maillage_3D(grandeur.min())}.2f}", f"{grandeur.max():{trouver_signe_afficher_maillage_3D(grandeur.max())}.2f}"],
                    )),
            showlegend = False #Evite d'avoir "Trace 0" etc. affichées à droite.
        ))
        #Création d'une trace affichant seulement les grandeurs négatives. C'est la trace(2)
        fig.add_trace(go.Scatter3d(
            x=X.flatten()[mask_charge_negative], y=Y.flatten()[mask_charge_negative], z=Z.flatten()[mask_charge_negative],
            mode="markers",
            marker=dict(size=2, #Taille des points.
                        color=grandeur.flatten()[mask_charge_negative],
                        colorscale=couleur,
                        cmin = grandeur.min(),
                        cmax = grandeur.max(),
                        opacity=1,
                        colorbar=dict(
                        title=f'{nom_grandeur} ({unite})', #Titre de la colorbar 
                        ticks="outside",  
                        tickvals=[grandeur.min(), grandeur.max()], #Plage des valeurs de la colorbar
                        ticktext=[f"{grandeur.min():{trouver_signe_afficher_maillage_3D(grandeur.min())}.2f}", f"{grandeur.max():{trouver_signe_afficher_maillage_3D(grandeur.max())}.2f}"],  
                    )),
            showlegend= False #Evite d'avoir "Trace 0" etc. affichées à droite.
        ))
        
        #Création de len(np.linspace(grandeur.min(),grandeur.max(),precision_slider)) traces affichant seulement une gamme de grandeur. Ce sont
        #les trace(3), ..., trace(len(np.linspace(grandeur.min(),grandeur.max(),precision_slider)) + 2).
        tolerance = float((grandeur.max() - grandeur.min())/(2*precision_slider))
        for valeur in np.linspace(grandeur.min(),grandeur.max(),precision_slider):
            fig.add_trace(go.Scatter3d(
            x=X.flatten()[abs(grandeur.flatten()-valeur) <= tolerance], y=Y.flatten()[abs(grandeur.flatten()-valeur) <= tolerance], z=Z.flatten()[abs(grandeur.flatten()-valeur) <= tolerance],
            mode="markers",
            marker=dict(size=2, #Taille des points.
                        color=grandeur.flatten()[abs(grandeur.flatten()-valeur) <= tolerance],
                        colorscale=couleur,
                        cmin = grandeur.min(),
                        cmax = grandeur.max(),
                        opacity=1,
                        ),
            showlegend= False #Evite d'avoir "Trace 0" etc. affichées à droite.
        ))
        
        #Création d'un slider permettant de mettre en avant un certain intervalle de valeurs.
        steps = []
        for valeur, charge in zip(range(precision_slider),np.linspace(grandeur.min(),grandeur.max(),precision_slider)):
            step = dict(
                method='update',
                args=[
                    {'marker.opacity': calcul_liste_arg_slider_afficher_maillage_3D(valeur,precision_slider)}, #Modifie l'opacité de la trace(1), ..., trace(max).
                ],
                label = f" {charge:.2f}{chr(177)}{tolerance:.3f} {unite}" #Légende du slider.
            )
            steps.append(step)
        slider = [dict(
            active = int(precision_slider/2), #Position intiale du slider.
            currentvalue = {'prefix' : f'Valeur {nom_grandeur} :'}, #Nom du slider.
            pad = {'t':50},
            steps = steps
        )]
        
        #Création d'un bouton permettant de réafficher toutes les valeurs.
        liste_arg_reset_button = [1,1] #Respectivement l'opacité des charges positives puis négatives.
        for _ in range(precision_slider):
            liste_arg_reset_button.append(0) #La valeur ajoutée dans la liste est l'opacité des traces des gammes des valeurs.
        
        reset_button = dict(
            type='buttons',
            showactive=True,
            buttons=[
                dict(label='Réinitialiser Opacité', #Nom du bouton.
                     method='update',
                     args=[{'marker.opacity': liste_arg_reset_button}] #Modifie l'opacité de la trace(1), ..., trace(max).
                )
            ],
            x=0.1, #Position en %
            y=0.9, #Position en % 
            xanchor="right",
            yanchor="bottom"
        )
        
        #Création d'un bouton qui permet de mettre en avant les valeurs positives.   
        liste_arg_bouton_valeur_positive = [1,0.1] #Respectivement l'opacité des charges positives puis négatives.
        for _ in range(precision_slider):
            liste_arg_bouton_valeur_positive.append(0) #La valeur ajoutée dans la liste est l'opacité des traces des gammes des valeurs.
     
        bouton_valeur_positive = dict(
            type='buttons',
            showactive=True,
            buttons=[
                dict(label=f'Afficher {nom_grandeur} +', #Nom du bouton.
                     method='update',
                     args=[{'marker.opacity': liste_arg_bouton_valeur_positive}] #Modifie l'opacité de la trace(1), ..., trace(max).
                )
            ],
            x=0.1, #Position en %
            y=0.7, #Position en %
            xanchor="right",
            yanchor="bottom"
        )
        
        #Création d'un bouton qui permet de mettre en avant les valeurs positives.   
        liste_arg_bouton_valeur_negative = [0.1,1] #Respectivement l'opacité des charges positives puis négatives.
        for _ in range(precision_slider):
            liste_arg_bouton_valeur_negative.append(0) #La valeur ajoutée dans la liste est l'opacité des traces des gammes des valeurs.
            
        bouton_valeur_negative = dict(
            type='buttons',
            showactive=True,
            buttons=[
                dict(label=f'Afficher {nom_grandeur} -', #Nom du bouton.
                     method='update',
                     args=[{'marker.opacity': liste_arg_bouton_valeur_negative}] #Modifie l'opacité de la trace(1), ..., trace(max).
                )
            ],
            x=0.1, #Position en %
            y=0.5, #Position en %
            xanchor="right",
            yanchor="bottom"
        )
        #Update et affiche le plot selon methode_affichage.
        fig.update_layout(
            title = titre,
            sliders = slider,
            updatemenus = [reset_button,bouton_valeur_positive,bouton_valeur_negative],
            scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
        )
        fig.show(methode_affichage)
    #Gère le cas dans lequel aucune valeur n'est affectée aux points.
    else:
        #Création d'une trace contenant tous les points du maillage.
        fig = go.Figure(data=[go.Scatter3d(
            x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
            mode="markers",
            marker=dict(size=2, opacity=0.8) 
        )])
        #Update et affiche le plot selon methode_affichage.
        fig.update_layout(
            title=titre,
            scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
        )
        fig.show(methode_affichage)
    #Gère le cas où l'on souhaite voir à quoi ressemblent les matrices du maillage X,Y,Z.
    if afficher_array:
        print(f"La première matrice ressemble à :\n\n{X},\n\n"
              f"La deuxième matrice ressemble à :\n\n{Y},\n\n"
              f"La dernière matrice ressemble à :\n\n{Z}")
    
    #C'est une méthode, elle ne retourne rien mais permet seulement d'afficher.
    return None

afficher_maillage_3D(X, Y, Z, "Visualisation du maillage")

In [30]:
def find_valeurs_point(x : float, y : float, z : float, X_maillage : np.array, Y_maillage : np.array, Z_maillage : np.array, X_grandeur : np.array, Y_grandeur = None, Z_grandeur = None, indice = None) -> np.array:
    """
    Entrées:
        
        - x: Représente la coordonnée selon x du point dont on souhaite connaître la/les valeur(s).
        - y: Représente la coordonnée selon y du point dont on souhaite connaître la/les valeur(s).
        - z: Représente la coordonnée selon z du point dont on souhaite connaître la/les valeur(s).
        
        - X_maillage: C'est une liste de NP.ARRAYS 2D qui décrit un maillage. Usuellement celle renvoyée par la fonction creer_maillage_3D() et celle dans laquelle se trouve le point choisi.
        - Y_maillage: C'est une liste de NP.ARRAYS 2D qui décrit un maillage. Usuellement celle renvoyée par la fonction creer_maillage_3D() et celle dans laquelle se trouve le point choisi.
        - Z_maillage: C'est une liste de NP.ARRAYS 2D qui décrit un maillage. Usuellement celle renvoyée par la fonction creer_maillage_3D() et celle dans laquelle se trouve le point choisi.
        
        - X_grandeur: C'est une liste de NP.ARRAYS 2D dont les dimensions sont identiques à celles du maillage. Par exemple, elle doit contenir nb_point_y NP.ARRAYS 2D de dimension (nb_point_x, nb_point_z).
            Elle associe à chaque point du maillage sa valeur. Si "Y_grandeur != None and Z_grandeur != None", représente la composante selon x du vecteur associé au point de l'espace.
        - Y_grandeur: C'est une liste de NP.ARRAYS 2D dont les dimensions sont identiques à celles du maillage. Par exemple, elle doit contenir nb_point_y NP.ARRAYS 2D de dimension (nb_point_x, nb_point_z).
            Elle représente la composante selon y du vecteur associé au point de l'espace.
        - Z_grandeur: C'est une liste de NP.ARRAYS 2D dont les dimensions sont identiques à celles du maillage. Par exemple, elle doit contenir nb_point_y NP.ARRAYS 2D de dimension (nb_point_x, nb_point_z).
            Elle représente la composante selon z du vecteur associé au point de l'espace.
            
        - indice: C'est un booléen. S'il vaut True, renvoie les indices auxquels se trouvent la/les valeur(s) dans X_grandeur (Y_grandeur et Z_grandeur).
            
    Sorties:
    
        - Si la valeur de sortie n'est pas un vecteur: Renvoie un NP.ARRAY 0D constitué de la valeur prise par le point dans X_grandeur.
        - Si la valeur de sorite est un vecteur: Renvoie un NP.ARRAY de taille (1x3) constitué des valeurs prises par le point dans respectivement X_grandeur, Y_grandeur et Z_grandeur.
        
    Explication:
    
        Renvoie la valeur ou le vecteur associée(s) au point considéré de l'espace dans la liste X_grandeur (Y_grandeur et Z_grandeur).
    """
    
    val_min_x = X_maillage[0][0,0]
    val_max_x = X_maillage[0][X_maillage.shape[0]-1,0]
    
    val_min_y = Y_maillage[0][0,0]
    val_max_y = Y_maillage[len(Y_maillage)-1][0,0]
    
    val_min_z = Z_maillage[0][0,0]
    val_max_z = Z_maillage[0][0,Z_maillage.shape[1]-1]
    
    nb_points_x = np.shape(X[0])[0]
    nb_points_y = len(X)
    nb_points_z = np.shape(X[0])[1]
    
    try:
        pas_suivant_x = float((int(val_max_x - val_min_x))  / (nb_points_x - 1))
        pas_suivant_y = float((int(val_max_y - val_min_y))  / (nb_points_y - 1))
        pas_suivant_z = float((int(val_max_z - val_min_z))  / (nb_points_z - 1))
    except:
        raise ZeroDivisionError()
    
    try:
        val_indice_x = int((x - val_min_x) / pas_suivant_x)
        val_indice_y = int((y - val_min_y) / pas_suivant_y)
        val_indice_z = int((z - val_min_z) / pas_suivant_z)
    except:
        raise ZeroDivisionError()
        
    if Y_grandeur is not None and Z_grandeur is not None:
        if indice:
            return (np.array([float(X_grandeur[val_indice_y][val_indice_x,val_indice_z]), float(Y_grandeur[val_indice_y][val_indice_x,val_indice_z]), float(Z_grandeur[val_indice_y][val_indice_x,val_indice_z])]),(val_indice_x,val_indice_y,val_indice_z))    
        return np.array([float(X_grandeur[val_indice_y][val_indice_x,val_indice_z]), float(Y_grandeur[val_indice_y][val_indice_x,val_indice_z]), float(Z_grandeur[val_indice_y][val_indice_x,val_indice_z])])    
    if indice:
        return (np.array([float(X_grandeur[val_indice_y][val_indice_x,val_indice_z])]), (val_indice_x,val_indice_y,val_indice_z))
    return np.array([float(X_grandeur[val_indice_y][val_indice_x,val_indice_z])])

print(find_valeurs_point(-10,-9.5,-9.5,X,Y,Z,X,Y,Z,True))

(array([-10. ,  -9.5,  -9.5]), (0, 1, 1))


In [ ]:
def calcul_charge_tout_point_maillage(X : np.array, Y : np.array, Z : np.array, fonction) -> tuple:
    """
    Entrées:
    
        - X: C'est une liste de NP.ARRAYS 2D. Usuellement celle renvoyée par la fonction creer_maillage_3D().
        - Y: C'est une liste de NP.ARRAYS 2D. Usuellement celle renvoyée par la fonction creer_maillage_3D().
        - Z: C'est une liste de NP.ARRAYS 2D. Usuellement celle renvoyée par la fonction creer_maillage_3D().
        
        - fonction: c'est une fonction définie par exemple via "lambda x,y,z : f(x,y,z)"
        
    Sortie:
    
        Renvoie une liste de NP.ARRAYS 2D de même dimension que le maillage. Elle contient les valeurs prises par le maillage pour chaque point de celui-ci.
        
    Explication:
    
        Renvoie les valeurs prises par le maillage selon la fonction prise en argument.
    """
    
    return fonction(X,Y,Z)

maillage_charge = calcul_charge_tout_point_maillage(X,Y,Z, lambda x,y,z : np.sin(x) + np.cos(y) + np.sin(z))

In [28]:
print(find_valeurs_point(0,1,-1,X,Y,Z,maillage_charge,None,None,True))

(array([-0.30116868]), (20, 22, 18))


In [8]:
afficher_maillage_3D(X,Y,Z, "Visualisation du maillage chargé", maillage_charge, "charge","V/m2")

In [8]:
def placer_charge(x : float, y : float, z : float, liste_charge_de_la_distribution : list) -> list:
    """
    Elle permet d'ajouter un np.array de coordonnées, représentant le fait qu'une charge y est placée, dans une liste décrivant la distribution
    """
    liste_charge_de_la_distribution.append(np.array([x,y,z], float))
    return liste_charge_de_la_distribution

test = list()

for i in range(2):
    for j in range(2):
        for k in range(2):
            placer_charge(i,j,k,test)

print(test)

[array([0., 0., 0.]), array([0., 0., 1.]), array([0., 1., 0.]), array([0., 1., 1.]), array([1., 0., 0.]), array([1., 0., 1.]), array([1., 1., 0.]), array([1., 1., 1.])]


In [9]:
test = list()

for i in range(2):
    for j in range(2):
        for k in range(2):
            placer_charge(i,j,k,test)

def deplacer_charge(x : float, y : float, z : float, dx : float, dy : float, dz : float, liste_charge_de_la_distribution) -> list:
    """
    La fonction prend les coordonnées du point (x,y,z) dans la liste_charge_de_la_distribution et les déplace respectivement de (dx,dy,dz).
    """
    for array in liste_charge_de_la_distribution:
        if np.array_equal(np.array([x,y,z],float),array):
            array[0] += dx
            array[1] += dy
            array[2] += dz

    return liste_charge_de_la_distribution

print(deplacer_charge(0,1,0,0.5,2,1,test))

[array([0., 0., 0.]), array([0., 0., 1.]), array([0.5, 3. , 1. ]), array([0., 1., 1.]), array([1., 0., 0.]), array([1., 0., 1.]), array([1., 1., 0.]), array([1., 1., 1.])]
